#### Imports/setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
import os
import geemap
import time
import functools
import os
import sys
# from datetime import datetime

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from parameters.config_runtime import (
    geo_id_column,
    out_directory,
    out_csv, 
    out_shapefile,
    cols_ind_1_treecover,
    cols_ind_2_commodities,
    cols_ind_3_dist_before_2020,
    cols_ind_4_dist_after_2020,
)

from modules.agstack_to_gee import (
    start_agstack_session,
    get_agstack_token,
    register_fc_and_append_to_csv,
    # add_geo_ids_to_csv_from_lookup_df,
    add_geo_ids_to_csv_from_lookup_csv,
    add_empty_column_to_csv,
    remove_column_from_csv,
    geo_id_or_ids_to_feature_collection
)

from parameters.config_asr_url_info import asset_registry_base, user_registry_base

from parameters.config_asr_credentials import email, password

from modules.utils import (
    collection_properties_to_df,
    remove_geometry_from_feature_collection,
    get_centroid
)

from modules.tidy_tables import whisp_risk, add_eudr_risk_col, add_indicators, select_years_in_range, create_wildcard_column_list

def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out


Earth Engine has been initialized with the specified credentials.


#### Set up asset registry

In [3]:
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)

#### Inputs : feature collection (without Geo IDs)

In [19]:
example_plot_feature_col_asset = 'projects/ee-andyarnellgee/assets/p0004_commodity_mapper_support/seg_civ_PCBM'

# Define the fraction of features you want to sample (e.g., 0.5 for 50%)
sampleFraction = 0.0030

In [20]:
fc = ee.FeatureCollection(example_plot_feature_col_asset)

randomColumn = 'random'
fc_r = fc.randomColumn(randomColumn,4)

# Filter the feature collection based on the random column to get the sample
fc_sample = fc_r.filter(ee.Filter.lt(randomColumn, sampleFraction));

roi = fc_sample

print(roi.size().getInfo())

roi


16


In [21]:
# temp code: uncomment below line if want to test using point inputs based on polygon centroid

# roi = roi.map(lambda feature: ee.Feature(feature.geometry().centroid(1))) 

### Whisp it
- If over >500 polygons currently this will go to Google Drive and require manual uploading here
- Stats sent to a separate notebook 'process_whisp_stats.pynb'

In [22]:
df = whisp_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Datasets present in lookup:  True
...stats processing started
...ee to df processing time: 7.181488037109375 seconds


### Display it


In [23]:
df

,Plot_area_ha,Geometry_type,Country,Centroid_lat,Centroid_lon,Unit,EUFO_2020,GLAD_Primary,TMF_undist,JAXA_FNF_2020,...,TMF_loss_before_2020,GFC_loss_before_2020,ESA_fire_before_2020,MODIS_fire_before_2020,TMF_deg_after_2020,TMF_loss_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index
0,0.385,Polygon,CIV,-3.695531,6.108556,ha,0.385,0,0.119,0.385,...,0.000,0.000,0,0,0.000,0.000,0.000,0,0,000000000000000007b6
1,0.702,Polygon,CIV,-3.649126,6.038068,ha,0.702,0,0.256,0.702,...,0.000,0.356,0,0,0.049,0.000,0.000,0,0,00000000000000000121
2,0.781,Polygon,CIV,-3.611461,6.122915,ha,0.503,0,0.139,0.781,...,0.089,0.129,0,0,0.000,0.000,0.000,0,0,000000000000000010e2
3,2.420,Polygon,CIV,-3.656310,6.110293,ha,2.035,0,1.630,1.649,...,0.356,0.870,0,0,0.405,0.335,0.108,0,0,0000000000000000023e
4,2.580,Polygon,CIV,-3.667554,6.085371,ha,2.095,0,0.455,2.580,...,0.148,0.722,0,0,0.030,0.000,0.138,0,0,000000000000000003d6
5,3.142,Polygon,CIV,-3.589833,6.113249,ha,3.142,0,2.548,3.142,...,0.000,0.297,0,0,0.089,0.000,0.000,0,0,00000000000000000df1
6,3.684,Polygon,CIV,-3.653701,6.115954,ha,2.626,0,0.089,3.487,...,0.118,0.860,0,0,0.000,0.079,0.188,0,0,000000000000000001ca
7,3.925,Polygon,CIV,-3.611783,6.106373,ha,3.925,0,1.999,3.925,...,0.000,1.076,0,0,0.000,0.000,0.000,0,0,0000000000000000110f
8,4.003,Polygon,CIV,-3.659884,6.030822,ha,3.273,0,0.880,3.924,...,0.198,0.572,0,0,0.000,0.000,0.000,0,0,0000000000000000029d
9,4.109,Polygon,CIV,-3.699359,6.101030,ha,4.109,0,2.156,4.109,...,0.000,0.594,0,0,0.000,0.000,0.000,0,0,00000000000000000835


#### Export it to CSV file
NB skip if exporting to Google Drive

In [24]:
#export output csv
print  (out_csv)
df.to_csv(path_or_buf=out_csv,header=True,index=False)

print ("output csv: ", out_csv)


/home/sepal-user/module_results/whisp/whisp_output_table.csv
output csv:  /home/sepal-user/module_results/whisp/whisp_output_table.csv


#### Add Geo ids
##### Step 1. Register polygons:  compiles geo_ids in a lookup csv
- NB registration takes a long time if many polygons (a few seconds each one). AgStack will update this in future. 
- In the meantime to stop timeouts extend session length click dollar sign and edit under 'Sessions'.
- If processing doesn't complete, rerun and it should pickup from where you got to.

In [25]:
register_fc_and_append_to_csv(
    feature_col=roi,
    geo_id_column="Geo_id",
    output_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
    join_id_column="system:index",
    token=token,
    session=session,
    asset_registry_base=asset_registry_base,
    override_checks=False,
    remove_temp_csv=False,
    debug=True)


Check passed: feature collection and table same size: 16 rows
Number without geo ids:0. 
 Processing started...
Done


##### Step 2. Join geo ids from lookup csv to Whisp stats csv

In [26]:
add_geo_ids_to_csv_from_lookup_csv(
        input_csv=out_directory/"whisp_output_table.csv",
        geo_id_lookup_csv=out_directory/"temp_geo_id_lookup.csv",
        join_id_column="system:index",
        geo_id_column=geo_id_column,
        overwrite=False,
        drop_geo=False,
        debug=True)

new csv:  /home/sepal-user/module_results/whisp/whisp_output_table_w_geo_id.csv


##### Optional: remove "system:index" column 
NB this is needed for joining geo_ids to csv (from lookup table). Check you have all your geo_ids first and if in doubt run on a copy of the results


In [27]:
# remove_column_from_csv(
#     csv_file="copy_whisp_output_table.csv", # this may change depending on if overwrite is on
#     column_name="system:index"
# )

##### Calculate EUDR risk category

In [28]:
# Replace with the name of your CSV file
csv_file =out_directory/'whisp_output_table_w_geo_id.csv'

# import as a dataframe
df_w_stats = pd.read_csv(csv_file)


df_w_risk = whisp_risk(
    df = df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )



##### Display table with risk columns

In [29]:
df_w_risk

,Plot_area_ha,Geo_id,Geometry_type,Country,Centroid_lat,Centroid_lon,Unit,EUFO_2020,GLAD_Primary,TMF_undist,...,TMF_loss_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index,Indicator_1_treecover,Indicator_2_commodities,Indicator_3_disturbance_before_2020,Indicator_4_disturbance_after_2020,EUDR_risk
0,0.385,eb7183ae1c62b147e099de79265607bf1a53cb4ff717d5...,Polygon,CIV,-3.695531,6.108556,ha,0.385,0,0.119,...,0.000,0.000,0,0,000000000000000007b6,yes,yes,yes,no,low
1,0.702,37485ae8d1dd2d047caa0fbc57a72e508e83b50bd3c4c1...,Polygon,CIV,-3.649126,6.038068,ha,0.702,0,0.256,...,0.000,0.000,0,0,00000000000000000121,yes,no,yes,yes,low
2,0.781,4b1ad3cd5dae97e1617b8cc7b39d2b5557f318abd8f47e...,Polygon,CIV,-3.611461,6.122915,ha,0.503,0,0.139,...,0.000,0.000,0,0,000000000000000010e2,yes,yes,yes,no,low
3,2.420,407e41306ce7edb11a28a3810c2db6b40fd0a4ab2d2e9a...,Polygon,CIV,-3.656310,6.110293,ha,2.035,0,1.630,...,0.335,0.108,0,0,0000000000000000023e,yes,no,yes,yes,low
4,2.580,049f0050aef223e4facfd31abd0b86de6f8181a2d76d87...,Polygon,CIV,-3.667554,6.085371,ha,2.095,0,0.455,...,0.000,0.138,0,0,000000000000000003d6,yes,yes,yes,yes,low
5,3.142,375ba910a586a125ff451e960451990123b31ba0e54fcd...,Polygon,CIV,-3.589833,6.113249,ha,3.142,0,2.548,...,0.000,0.000,0,0,00000000000000000df1,yes,yes,yes,yes,low
6,3.684,ed21ceabff267e7017b346b4a0afa47f29af1177b41194...,Polygon,CIV,-3.653701,6.115954,ha,2.626,0,0.089,...,0.079,0.188,0,0,000000000000000001ca,yes,no,yes,yes,low
7,3.925,a56bc7919b0a7935552df735c1db96d0fb41202df4d944...,Polygon,CIV,-3.611783,6.106373,ha,3.925,0,1.999,...,0.000,0.000,0,0,0000000000000000110f,yes,no,yes,no,low
8,4.003,146d51ab4e6a931f392efde0b270a3865e6d59a8eea559...,Polygon,CIV,-3.659884,6.030822,ha,3.273,0,0.880,...,0.000,0.000,0,0,0000000000000000029d,yes,yes,yes,no,low
9,4.109,b98c8bc02d5057ad4f2a1ceea4fa235af0d1333703ffae...,Polygon,CIV,-3.699359,6.101030,ha,4.109,0,2.156,...,0.000,0.000,0,0,00000000000000000835,yes,yes,yes,no,low


##### Export table with risk columns to csv 

In [30]:
output_file=out_directory/'whisp_output_table_w_geo_id_w_risk.csv'

# export to a csv
df_w_risk.to_csv(output_file)
